In [2]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from scipy.spatial import cKDTree
import trimesh
import numpy as np

In [3]:
def map_vertices(source_mesh, target_mesh, normalize=False):
    """
    Get closest vertex index on source mesh
    Args:
        source_mesh (trimesh.Trimesh)
        target_mesh (trimesh.Trimesh)
    Returns:
        distances (np.ndarray): distance with the closest vertex on source mesh
        vertex_map (np.ndarray): index of the closest vertex on source mesh
    """
    src_v = source_mesh.vertices
    tgt_v = target_mesh.vertices
    
    if normalize:
        src_v = normalize_mesh(src_v)
        tgt_v = normalize_mesh(tgt_v)
        
    tree = cKDTree(src_v)
    distances, vertex_map = tree.query(tgt_v, k=1)
    return distances, vertex_map

def normalize_mesh(vertices, mode=1):
    """
    Args:
        vertices (np.ndarray): raw vertex position
        mode (bool): if True, apply rescaling
        # mode==1: range in (-1 ~ 1) || mode==0: range in (0 ~ 1)
    Return:
        vertices (np.ndarray): normalized vertex position
    """
    # Center the vertices at the origin
    vertices = vertices - (vertices.max(0)+vertices.min(0))*0.5
    
    # Find the maximum range in any dimension
    max_range = np.max(np.ptp(vertices, axis=0))

    # Scale the vertices to fit within the -1 to 1 range while maintaining aspect ratio
    if mode:
        vertices = vertices / max_range * 2
    return vertices

def compute_features(device, pipe, dino_model, m, prompt):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]
    features = get_features_per_vertex(
        device=device,
        pipe=pipe, 
        dino_model=dino_model,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        num_images_per_prompt=num_images_per_prompt,
        use_normal_map=use_normal_map,
    )
    return features.cpu()

In [4]:
# mesh_name = 'AlienSoldier'
#mesh_name = 'Amy'
# mesh_name = 'Astra'
# mesh_name = 'Brady'
# mesh_name = 'Brian'
# mesh_name = 'Bryce'
# mesh_name = 'CastleGuard01'
# mesh_name = 'Chad'
# mesh_name = 'Claire'
# mesh_name = 'CopzombieLActisdato'
# mesh_name = 'Jolleen'
# mesh_name = 'Kaya'
# mesh_name = 'MariaJJOng'
# mesh_name = 'MremirehODesbiens'
# mesh_name = 'Ortiz'
# mesh_name = 'PumpkinhulkL'
# mesh_name = 'TheBoss'
# mesh_name = 'Timmy'
# mesh_name = 'Ty'
mesh_name = 'VanguardByTChoonyung'

In [5]:

base_path = '../../../SAUS_prev/Diffusion-3D-Features/Mesh_data'
ee = np.load(f'{base_path}/{mesh_name}/Mesh/{mesh_name}.npz')

print(ee.__dict__)

print(ee['feat'].shape)
print(ee['vert'].shape)

# np.save(f'{base_path}/{mesh_name}/Mesh/tpose_mesh_diff3f',ee['feat'])

mm = np.load(f'{base_path}/{mesh_name}/Mesh/tpose_mesh.npz')
print(mm.__dict__)
print(mm['c_rest_pos'].shape)

ee_mesh = trimesh.Trimesh(vertices=ee['vert'], faces=ee['face'], maintain_order=True, process=False)
mm_mesh = trimesh.Trimesh(vertices=mm['c_rest_pos'], faces=mm['fid_to_cids'], maintain_order=True, process=False)

_ =ee_mesh.export('./source_mesh.obj')
_ =mm_mesh.export('./target_mesh.obj')

FileNotFoundError: [Errno 2] No such file or directory: '../../../SAUS_prev/Diffusion-3D-Features/Mesh_data/VanguardByTChoonyung/Mesh/VanguardByTChoonyung.npz'

In [6]:
device = torch.device('cuda:0')

f_source = torch.from_numpy(ee['feat'])
f_target = f_source
source_mesh = MeshContainer().load_from_file('./source_mesh.obj')
target_mesh = MeshContainer().load_from_file('./target_mesh.obj')
s = cosine_similarity(f_source.to(device), f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert)
cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

NameError: name 'ee' is not defined

In [7]:
distance, vertex_map = map_vertices(ee_mesh, mm_mesh,normalize=True)
print(vertex_map)
f_source = torch.from_numpy(ee['feat'])
f_target = torch.from_numpy(ee['feat'][vertex_map])
source_mesh = MeshContainer().load_from_file('./source_mesh.obj')
target_mesh = MeshContainer().load_from_file('./target_mesh.obj')
s = cosine_similarity(f_source.to(device), f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert)
cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)

# np.save(f'{base_path}/{mesh_name}/Mesh/tpose_mesh_diff3f', ee['feat'][vertex_map])
# np.save(f'{base_path}/{mesh_name}/Mesh/tpose_mesh_diff3f_src', ee['feat'])

NameError: name 'ee_mesh' is not defined